In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from time import sleep
from processor.processor import Processor as p
from extractor.tiingo_extractor import TiingoExtractor
from extractor.fred_extractor import FREDExtractor

In [2]:
market = Market()
sp500 = ADatabase("sp500")
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
market.connect()
start = market.retrieve_max_date("stocks")
market.disconnect()
end = datetime.now().strftime("%Y-%m-%d")

In [4]:
analysis = []
market.connect()
periods = ["year","quarter","week"]
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("stocks",ticker)
        test = p.column_date_processing(test)
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["rolling_100"] = test["adjclose"].rolling(window=100).mean()
        ep = test.tail(1)["adjclose"].item()
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjclose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjclose"].item())/test.tail(2).iloc[0]["adjclose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:32<00:00, 15.09it/s]


In [5]:
period = "day"
a = pd.DataFrame(analysis)
a = a.merge(sp5[["Symbol","Security","GICS Sector","GICS Sub-Industry"]].rename(columns={"Symbol":"ticker"}),on="ticker")
a.sort_values(f"{period}_gain",ascending=False).head(20)

,year_gain,quarter_gain,week_gain,day_gain,ticker,Security,GICS Sector,GICS Sub-Industry
482,0.250905,0.004408,0.0,0.056948,WYNN,Wynn Resorts,Consumer Discretionary,Casinos & Gaming
167,-0.307291,0.034916,0.0,0.055967,ENPH,Enphase,Information Technology,Electronic Components
302,0.367187,0.038813,0.0,0.055929,MGM,MGM Resorts,Consumer Discretionary,Casinos & Gaming
381,0.187746,0.040909,0.0,0.050592,RL,Ralph Lauren Corporation,Consumer Discretionary,"Apparel, Accessories & Luxury Goods"
354,-0.114110,0.068397,0.0,0.048524,PENN,Penn National Gaming,Consumer Discretionary,Casinos & Gaming
236,0.283737,0.063303,0.0,0.046585,IDXX,Idexx Laboratories,Health Care,Health Care Equipment
19,0.619088,-0.002583,0.0,0.042969,ALGN,Align Technology,Health Care,Health Care Supplies
197,0.604699,0.042861,0.0,0.042582,FTNT,Fortinet,Information Technology,Systems Software
312,0.561170,-0.015187,0.0,0.039755,MPWR,Monolithic Power Systems,Information Technology,Semiconductors
376,0.219463,-0.004464,0.0,0.039312,PVH,PVH,Consumer Discretionary,"Apparel, Accessories & Luxury Goods"


In [6]:
a.groupby("GICS Sector").mean().sort_values("quarter_gain",ascending=False).head(20)

,year_gain,quarter_gain,week_gain,day_gain
GICS Sector,,,,
Energy,-0.014639,0.012083,0.000000,0.005175
Industrials,0.159290,0.003910,0.000000,0.015341
Information Technology,0.220172,0.003030,0.000000,0.018201
Consumer Discretionary,0.177466,0.001320,0.000000,0.018549
Communication Services,0.132015,0.000580,0.001840,-0.002759
Real Estate,0.027209,-0.001211,0.000000,0.004032
Health Care,0.018991,-0.003924,-0.001451,0.011381
Financials,-0.048738,-0.004098,0.000444,0.003999
Utilities,-0.044208,-0.009521,0.000000,-0.004578
